In [452]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import random

import matplotlib.pyplot as plt
import seaborn as sns

In [453]:
import yfinance as yf
import requests

In [454]:
s = '2023-11-01'
e = '2023-12-31'

In [455]:
df = pd.read_csv("https://query.data.world/s/pzd7uazpmpyortmg6fsuxiaoizllgu?dws=00000", encoding='ISO-8859-1')
display(df.head())

,Public Company Name (As Listed on Exchange),B Corp Impact Data ID,Ticker 1,Ticker 2,Ticker 3,IPO Year,Year Delisted,Year Certified,Public Listing Status,Year Decertified
0,"Allbirds, Inc.",001C000001SGhnyIAD,NASDAQGS:BIRD,NaN,NaN,2021.0,NaN,2016.0,active,NaN
1,Amalgamated Financial,001C000001K16JSIAZ,NASDAQGM:AMAL,NaN,NaN,2018.0,NaN,2017.0,active,NaN
2,"AppHarvest, Inc.",0013b00001o41JFAAY,NASDAQGS:APPH,NaN,NaN,2020.0,NaN,2019.0,delisted,2023.0
3,Arezzo&Co,0013b00001qHFddAAG,BOVESPA:ARZZ3,NaN,NaN,2011.0,NaN,2022.0,active,NaN
4,Arowana International,001C000001ZzPK4IAN,ASX: AWN,NaN,NaN,2013.0,2021.0,2018.0,delisted,NaN


In [456]:
df.drop(['B Corp Impact Data ID', 'Ticker 2', 'Ticker 3'], axis=1, inplace = True)
print(len(df))
df.dropna(axis=0, subset=['Public Company Name (As Listed on Exchange)'], inplace=True)
print(len(df))

999
68


In [457]:
display(df[df['Public Listing Status'] == 'delisted'])

,Public Company Name (As Listed on Exchange),Ticker 1,IPO Year,Year Delisted,Year Certified,Public Listing Status,Year Decertified
2,"AppHarvest, Inc.",NASDAQGS:APPH,2020.0,NaN,2019.0,delisted,2023.0
4,Arowana International,ASX: AWN,2013.0,2021.0,2018.0,delisted,NaN
14,Freshlocal Solutions,TSE: LOCL,2019.0,2021.0,2013.0,delisted,NaN


In [458]:
temp = df.copy()
temp = temp[temp['Public Listing Status'] == 'active']

temp['Ticker 1'].replace(": ", ":", regex=True, inplace=True)


print(len(temp))
temp['Exchange'] = temp['Ticker 1'].str.split(":").str[0]
temp['Ticker2'] = temp['Ticker 1'].str.split(":").str[1]
temp.dropna(subset=["Ticker2"], inplace=True) # Remove any NaN tickers

display(temp.head())

65


,Public Company Name (As Listed on Exchange),Ticker 1,IPO Year,Year Delisted,Year Certified,Public Listing Status,Year Decertified,Exchange,Ticker2
0,"Allbirds, Inc.",NASDAQGS:BIRD,2021.0,NaN,2016.0,active,NaN,NASDAQGS,BIRD
1,Amalgamated Financial,NASDAQGM:AMAL,2018.0,NaN,2017.0,active,NaN,NASDAQGM,AMAL
3,Arezzo&Co,BOVESPA:ARZZ3,2011.0,NaN,2022.0,active,NaN,BOVESPA,ARZZ3
5,Australian Ethical Investment Ltd.,ASX:AEF,2002.0,NaN,2014.0,active,NaN,ASX,AEF
6,Benefit Systems S.A.,WSE:BFT,2013.0,NaN,2018.0,active,NaN,WSE,BFT


### Stocks are from Global Exchanges - build and maintain an exchange conversion dictionary to interface with Yahoo Finance

In [459]:
exchange_dict = {'ASX': '.XA', 
                 'WSE': '.WA',
                 'BOVESPA': '.SA',
                 'ETHEX': '',
                 'TSX': '.TO',
                 'ENXTPA': '.PA',
                 'BME': '.MC',
                 'ENXTBR': '.BR',
                 'KLSE': '.KL',
                 'LSE': '.AQ',
                 'TWSE': '.TW',
                 'NZSE': '.NZ',
                 'SGX': '.SI',
                 'BIT': '.MI',
                 'TASE': '.TA',
                 'XMIL': '.MI',
                 'NZE': '.NZ',
                 'BVMF': '.SA',
                 'HKG': '.HK',
                 'LON': '.L',
                 'MIL': '.MI',
                 'MOVI3': '.SA',
                 'TSE': '.T',
                 'BRK': '.L',
                 'SNSE': '.SN',
                 'MOVIDA B3': '.SA',
                 'NYSE': '',
                 'NASDAQGS': '',
                 'NASDAQGM': '',
                 'NASDAQCM': ''
                }

exchange_map = {exchanges: extention
                 for exchanges, extention in exchange_dict.items()
                 for exchange in exchanges}

temp['Intl Extention'] = temp['Exchange'].map(exchange_map)

In [460]:
temp['Ticker'] = temp['Ticker2'] + temp['Intl Extention'] 
mask = temp['Ticker'].isna()

# Check to see if we missed any:
temp[mask]

,Public Company Name (As Listed on Exchange),Ticker 1,IPO Year,Year Delisted,Year Certified,Public Listing Status,Year Decertified,Exchange,Ticker2,Intl Extention,Ticker
19,Heidelberg District Community Enterprise Limited,NSX:EIC,2019.0,NaN,2019.0,active,NaN,NSX,EIC,NaN,NaN
36,SIDERPERU,BVL:SIDERC1,2006.0,NaN,2022.0,active,NaN,BVL,SIDERC1,NaN,NaN
41,Urb-it AB (publ),OM:URBIT,2017.0,NaN,2020.0,active,NaN,OM,URBIT,NaN,NaN
48,Weleda AG Group,OTC-X:BEKB:Weleda N-PS,NaN,NaN,2021.0,active,NaN,OTC-X,BEKB,NaN,NaN
56,Vinedos Emiliana S.A.,XSGO:EMILIANA,1996.0,NaN,2023.0,active,NaN,XSGO,EMILIANA,NaN,NaN


In [461]:
#Find datafeed issues
tickers = temp['Ticker'].to_list()

count = 0
for t in tickers:
    try:
        ticker = yf.Ticker(t)
        info = None
    except:
        count +=1
        print(f"Cannot get info of {t}, it probably does not exist")

print("Total nans =", count, "which is", round(count/temp.shape[0],2)*100, '% of the total')


Cannot get info of nan, it probably does not exist
Cannot get info of nan, it probably does not exist
Cannot get info of nan, it probably does not exist
Cannot get info of nan, it probably does not exist
Cannot get info of nan, it probably does not exist
Total nans = 5 which is 8.0 % of the total


In [462]:
# Remove Nans from stock universe:

b_corp_df = temp[-mask].copy()
display(b_corp_df.head())


,Public Company Name (As Listed on Exchange),Ticker 1,IPO Year,Year Delisted,Year Certified,Public Listing Status,Year Decertified,Exchange,Ticker2,Intl Extention,Ticker
0,"Allbirds, Inc.",NASDAQGS:BIRD,2021.0,NaN,2016.0,active,NaN,NASDAQGS,BIRD,,BIRD
1,Amalgamated Financial,NASDAQGM:AMAL,2018.0,NaN,2017.0,active,NaN,NASDAQGM,AMAL,,AMAL
3,Arezzo&Co,BOVESPA:ARZZ3,2011.0,NaN,2022.0,active,NaN,BOVESPA,ARZZ3,.SA,ARZZ3.SA
5,Australian Ethical Investment Ltd.,ASX:AEF,2002.0,NaN,2014.0,active,NaN,ASX,AEF,.XA,AEF.XA
6,Benefit Systems S.A.,WSE:BFT,2013.0,NaN,2018.0,active,NaN,WSE,BFT,.WA,BFT.WA


In [463]:
isolate_us = False
if isolate_us:

    exchanges = set(temp['Exchange'])
    print(exchanges)
    us_ex = ['NYSE', 'NASDAQGS','NASDAQGM', 'NASDAQCM']
    us = temp[temp['Exchange'].isin(us_ex)]
    print("Isolating US Companies")
    display(us.head())


### List of Companies with B-Corp Subsidiaries
Unilever (NYSE: UN): (Pukka (UK), Mãe Terra (Brazil), Sir Kensington’s (US), Ben & Jerry’s (US), Olly Nutrition (US), Seventh Generation (US), Sundial (US), T2 Tea (Australia), Graze (UK))

B2W (B3: BTOW3): Courrieros (Brazil)

Grupo Soma (B3: SOMA3): Companhia Hering (Brazil)

Arezzo&Co (B3: ARZZ3): Reserva (Brazil)

Procter & Gamble (NYSE: PG): (New Chapter (US))

AB Inbev: (4 Pines Brewing Company)

Banco Estado: (Banco Estado Microempresas, Caja Vecina)

Coca-Cola: (Innocent Drinks)

Fairfax Financial: (The Redwoods Group)

Gap: (Athleta, Hill City)

Kikkoman: (Country Life)

Lactalis: (Stonyfield Farm)

Land O’ Lakes: (Vermonta Creamery)

Nestle: (Essential Living Foods, Garden of Life, Lily's Kitchen)

Oppenheimer Funds Inc.: (SNW Asset Management)

Rakuten: (OverDrive)

The Hain Celestial Group: (Ella’s Kitchen)

Vina Concha y Toro: (Fetzer Vineyards)


Hortfruti S.A.: Hortifruti Chile

Danone SA: (Danone North America, Alpro, Danone Waters of America, Danone Canada, -Aguas Danone Argentina, Bledina, Danone Aqua Indonesia, Danone Egypt, Danone Iberia, Danone Manifesto Ventures, Danone UK, Grameen Danone Foods, Happy Family, Les 2 Vaches, Danone Waters Germany, Danone Dairy Ireland, Danone Waters Spain, Danone Netherlands, Danone Dairy Belgium, Nutricia Bago, Danone ELN Greater China, Danone Japan, Volvic, Danone Brazil)

Caixabank (Imagin Ban)

Banco Sabadell (Sogeviso)

iSun (SunCommon)

In [464]:
add_subs = True
public_subs = ['UL', 'SOMA3.SA', 'ABI.BR', 'PG', 'FFH.TO', 'KO', 'GPS', 'KIK.F', 'NESN.SW',
               'OPY', '4755.T', 'HAIN', 'DANOY', 'CABK.MC', 'ISUN']

In [465]:
def get_stock_data (df):
    tickers_list = df['Ticker'].tolist()
    if add_subs:
        tickers_list = tickers_list + public_subs
    tickers_data= {}
    
    for ticker in tickers_list:
        ticker_object = yf.Ticker(ticker)
        
        #convert info() output from dictionary to dataframe
        temp = pd.DataFrame.from_dict(ticker_object.info, orient="index")
        temp.reset_index(inplace=True)
        temp.columns = ["Attribute", "Recent"]

        # add (ticker, dataframe) to main dictionary
        tickers_data[ticker] = temp

    return tickers_data
        
        

In [466]:
if isolate_us:
    load_stock_data = get_stock_data (us)
else:
    load_stock_data = get_stock_data (b_corp_df)
    
print("Number of Stock Records Loaded", len(load_stock_data))

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SZL.XA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SZL.XA&crumb=FUh37WBIK1d


Number of Stock Records Loaded 71


In [467]:
# Clean df:
stock_data = pd.concat(load_stock_data) #turn from dict to df
stock_data = stock_data.reset_index()

del stock_data["level_1"] # clean up unnecessary column
stock_data.columns = ["Ticker", "Attribute", "Recent"] # update column names


attr_to_keep = ['shortName', 'longName', 'industry', 'sector','previousClose', 'sharesOutstanding', 'impliedSharesOutstanding',
               'marketCap',  'currency', 'country', 'website', 'enterpriseValue',
                'trailingEps', 'forwardEps', 'enterprisetoEbitda', 'ebitda', 'grossMargins', 'ebitdaMargins', 'operatingMargins']


stock_data = stock_data[stock_data["Attribute"].isin(attr_to_keep)].reset_index()
del stock_data["index"] # clean up unnecessary column



display(stock_data.head(15))

,Ticker,Attribute,Recent
0,BIRD,country,United States
1,BIRD,website,https://www.allbirds.com
2,BIRD,industry,Apparel Retail
3,BIRD,sector,Consumer Cyclical
4,BIRD,previousClose,0.6188
5,BIRD,marketCap,96651704
6,BIRD,currency,USD
7,BIRD,enterpriseValue,59948084
8,BIRD,sharesOutstanding,102641000
9,BIRD,impliedSharesOutstanding,155188992


In [468]:
print("Number of Stocks:", len(stock_data['Ticker'].unique()))
display(stock_data['Ticker'].unique())

Number of Stocks: 68


array(['BIRD', 'AMAL', 'ARZZ3.SA', 'AEF.XA', 'BFT.WA', 'BIO.XA',
       'CWEB.TO', 'CLSA3.SA', 'MLMFI.PA', 'COUR', 'FLOW.TO', 'FREY.PA',
       'GFI.NZ', 'GROV', 'HAN.MC', 'HLZ.MC', 'INCLU.BR', 'IBAB.BR',
       'KPG.XA', 'KCT.AQ', 'KMD.NZ', 'LMND', 'LFG.XA', 'MOVI3.SA',
       'NTCO3.SA', '2897.TW', '1H2.SI', 'ALQP.PA', 'RETI.MI', '6088.T',
       'SML.NZ', 'COCO', 'VNT.MI', 'CONCHATORO.SN', 'VITL', 'VVPR',
       'VHT.XA', 'WRBY', 'ZVIA', 'ABB.XA', 'TIME.MI', 'CVT.NZ', 'DOX.MI',
       'SOMA3.SA', '1112.HK', 'HMGS.TA', '1346.HK', '0973.HK', 'MME.XA',
       'MPR.XA', 'TENG.L', 'TPX.L', 'XSG.L', 'XHS.MI', 'UL', 'ABI.BR',
       'PG', 'FFH.TO', 'KO', 'GPS', 'KIK.F', 'NESN.SW', 'OPY', '4755.T',
       'HAIN', 'DANOY', 'CABK.MC', 'ISUN'], dtype=object)

In [469]:
stock_data_filtered = stock_data.pivot(columns = 'Attribute', index='Ticker', values='Recent')
stock_data_filtered.columns

# More info:
#stock_data_filtered = stock_data_filtered[['shortName', 'longName','country', 'currency', 'previousClose',
#                                           'sharesOutstanding', 'impliedSharesOutstanding','marketCap', 'enterpriseValue',
#                                           'ebitda', 'forwardEps',  'trailingEps', 'grossMargins', 'operatingMargins', 'ebitdaMargins',
#                                             'industry', 'sector', 'website']]
# Only essential info:
stock_data_filtered = stock_data_filtered[['shortName', 'longName','country', 'currency', 'previousClose',
                                           'impliedSharesOutstanding','marketCap','industry', 'sector', 'website']]

display(stock_data_filtered.head(10))

Attribute,shortName,longName,country,currency,previousClose,impliedSharesOutstanding,marketCap,industry,sector,website
Ticker,,,,,,,,,,
0973.HK,L'OCCITANE,L'Occitane International S.A.,Luxembourg,HKD,31.2,1472169984,46520573952,Household & Personal Products,Consumer Defensive,https://group.loccitane.com
1112.HK,H&H INTL HLDG,Health and Happiness (H&H) International Holdi...,Hong Kong,HKD,11.14,645561024,7088259584,Packaged Foods,Consumer Defensive,https://www.hh.global
1346.HK,LEVER STYLE,Lever Style Corporation,Hong Kong,HKD,0.95,645899008,607145024,Apparel Manufacturing,Consumer Cyclical,https://www.leverstyle.com
1H2.SI,$ Olive Tree,Olive Tree Estates Limited,Singapore,SGD,0.078,124756000,9035910,Real Estate - Diversified,Real Estate,https://www.olivetreeestates.com
2897.TW,O-BANK CO LTD,"O-Bank Co., Ltd.",Taiwan,TWD,9.9,2716470016,27164700672,Banks - Regional,Financial Services,https://www.o-bank.com
4755.T,RAKUTEN GROUP INC,"Rakuten Group, Inc.",Japan,JPY,879.1,2142140032,1854664867840,Internet Retail,Consumer Cyclical,https://global.rakuten.com/corp
6088.T,SIGMAXYZ HOLDINGS INC,SIGMAXYZ Holdings Inc.,Japan,JPY,1716.0,42581400,73618423808,Specialty Business Services,Industrials,https://www.sigmaxyz.com
ABB.XA,AUSSIEBAND FPO,Aussie Broadband Limited,Australia,AUD,3.6,NaN,NaN,NaN,NaN,https://www.aussiebroadband.com.au
ABI.BR,AB INBEV,Anheuser-Busch InBev SA/NV,Belgium,EUR,55.82,282105984,110587289600,Beverages - Brewers,Consumer Defensive,https://www.ab-inbev.com


### Incorporate exchange rates to translate price and market cap into USD

In [470]:
# Get avaialable currencies
currency_dict = {}

url = "https://currency-conversion-and-exchange-rates.p.rapidapi.com/symbols"

    #headers = {"X-RapidAPI-Key": "af4b092441msh3f4304a0bce0366p1680a3jsn93d4c3a0659f",
headers = {"X-RapidAPI-Key": "884638c782msh3dc9c5784730cc0p1a0f42jsnf54c2327e84c",
    "X-RapidAPI-Host": "currency-conversion-and-exchange-rates.p.rapidapi.com"
          }

response = requests.get(url, headers=headers)

api_data = response.json()
#print(api_data)

    
for symbol, curr in api_data['symbols'].items():
    currency_dict[symbol] = [curr, 0]

#currency_dict
    

In [471]:
## Access Exchange Rates:

url = "https://currency-conversion-and-exchange-rates.p.rapidapi.com/latest"

#querystring = {"from":"USD","to":"EUR,GBP"}
querystring = {'base':'USD', 'date': '2024-02-01'}

    #headers = {"X-RapidAPI-Key": "af4b092441msh3f4304a0bce0366p1680a3jsn93d4c3a0659f",
headers = {"X-RapidAPI-Key": "884638c782msh3dc9c5784730cc0p1a0f42jsnf54c2327e84c",
    "X-RapidAPI-Host": "currency-conversion-and-exchange-rates.p.rapidapi.com"
          }

response = requests.get(url, headers=headers, params=querystring)

exch_data = response.json()

for key in exch_data: 
    print(key,":", exch_data[key]) 

timestamp : 1711555684
base : USD
success : True
date : 2024-03-27
rates : {'ANG': 1.80267, 'SVC': 8.751594, 'CAD': 1.358965, 'XCD': 2.70255, 'MVR': 15.450114, 'HRK': 6.882375, 'AUD': 1.53245, 'MWK': 1739.999587, 'XAG': 0.040779, 'MAD': 10.118502, 'PHP': 56.2625, 'NAD': 18.980276, 'GNF': 8594.999939, 'KES': 130.999758, 'MZN': 63.521876, 'BTN': 83.358137, 'MGA': 4389.999547, 'AZN': 1.699294, 'XAU': 0.000456, 'RON': 4.594897, 'CHF': 0.905735, 'EGP': 47.250402, 'BSD': 1.000222, 'TWD': 31.995498, 'GGP': 0.791928, 'LVL': 0.60489, 'MMK': 2100.456621, 'WST': 2.766241, 'ILS': 3.676705, 'BHD': 0.376959, 'GBP': 0.79173, 'TZS': 2551.999896, 'SDG': 600.999924, 'LAK': 20950.000029, 'DJF': 178.109922, 'BYN': 3.272789, 'LBP': 89750.000016, 'RWF': 1288, 'PEN': 3.729736, 'EUR': 0.923955, 'ZMK': 9001.206879, 'RSD': 108.276012, 'INR': 83.32845, 'MUR': 46.289626, 'BWP': 13.710646, 'GEL': 2.710217, 'KMF': 454.525034, 'UZS': 12615.000444, 'RUB': 92.484979, 'CUC': 1, 'BGN': 1.806393, 'JOD': 0.708899, 'NGN': 

In [472]:
## Combine date from exch_data with names / symbols in currency_dict:

for symbol, rate in exch_data['rates'].items():
    for key in currency_dict.keys():
        if symbol == key:
            currency_dict[key][1] = rate

In [473]:
currency_dict

{'ANG': ['Netherlands Antillean Guilder', 1.80267],
 'SVC': ['Salvadoran Colón', 8.751594],
 'CAD': ['Canadian Dollar', 1.358965],
 'XCD': ['East Caribbean Dollar', 2.70255],
 'MVR': ['Maldivian Rufiyaa', 15.450114],
 'HRK': ['Croatian Kuna', 6.882375],
 'AUD': ['Australian Dollar', 1.53245],
 'MWK': ['Malawian Kwacha', 1739.999587],
 'XAG': ['Silver (troy ounce)', 0.040779],
 'MAD': ['Moroccan Dirham', 10.118502],
 'PHP': ['Philippine Peso', 56.2625],
 'NAD': ['Namibian Dollar', 18.980276],
 'GNF': ['Guinean Franc', 8594.999939],
 'KES': ['Kenyan Shilling', 130.999758],
 'MZN': ['Mozambican Metical', 63.521876],
 'BTN': ['Bhutanese Ngultrum', 83.358137],
 'MGA': ['Malagasy Ariary', 4389.999547],
 'AZN': ['Azerbaijani Manat', 1.699294],
 'XAU': ['Gold (troy ounce)', 0.000456],
 'RON': ['Romanian Leu', 4.594897],
 'CHF': ['Swiss Franc', 0.905735],
 'EGP': ['Egyptian Pound', 47.250402],
 'BSD': ['Bahamian Dollar', 1.000222],
 'TWD': ['New Taiwan Dollar', 31.995498],
 'GGP': ['Guernsey Po

In [474]:
## Add exhange rates and calculate USD equivalents:

#exchange_map = {exchanges: extention
#                 for exchanges, extention in exchange_dict.items()
#                 for exchange in exchanges}


currency_map = {symbols: rate
                 for symbols, name in currency_dict.items()
                 for rate in name}



stock_data_filtered['USDExchangeRate'] = stock_data_filtered['currency'].map(currency_map)



In [475]:
# Check to see that exchange rates mapped correctly:
stock_data_filtered.head(15)

Attribute,shortName,longName,country,currency,previousClose,impliedSharesOutstanding,marketCap,industry,sector,website,USDExchangeRate
Ticker,,,,,,,,,,,
0973.HK,L'OCCITANE,L'Occitane International S.A.,Luxembourg,HKD,31.2,1472169984,46520573952,Household & Personal Products,Consumer Defensive,https://group.loccitane.com,7.823499
1112.HK,H&H INTL HLDG,Health and Happiness (H&H) International Holdi...,Hong Kong,HKD,11.14,645561024,7088259584,Packaged Foods,Consumer Defensive,https://www.hh.global,7.823499
1346.HK,LEVER STYLE,Lever Style Corporation,Hong Kong,HKD,0.95,645899008,607145024,Apparel Manufacturing,Consumer Cyclical,https://www.leverstyle.com,7.823499
1H2.SI,$ Olive Tree,Olive Tree Estates Limited,Singapore,SGD,0.078,124756000,9035910,Real Estate - Diversified,Real Estate,https://www.olivetreeestates.com,1.348085
2897.TW,O-BANK CO LTD,"O-Bank Co., Ltd.",Taiwan,TWD,9.9,2716470016,27164700672,Banks - Regional,Financial Services,https://www.o-bank.com,31.995498
4755.T,RAKUTEN GROUP INC,"Rakuten Group, Inc.",Japan,JPY,879.1,2142140032,1854664867840,Internet Retail,Consumer Cyclical,https://global.rakuten.com/corp,151.355498
6088.T,SIGMAXYZ HOLDINGS INC,SIGMAXYZ Holdings Inc.,Japan,JPY,1716.0,42581400,73618423808,Specialty Business Services,Industrials,https://www.sigmaxyz.com,151.355498
ABB.XA,AUSSIEBAND FPO,Aussie Broadband Limited,Australia,AUD,3.6,NaN,NaN,NaN,NaN,https://www.aussiebroadband.com.au,1.532450
ABI.BR,AB INBEV,Anheuser-Busch InBev SA/NV,Belgium,EUR,55.82,282105984,110587289600,Beverages - Brewers,Consumer Defensive,https://www.ab-inbev.com,0.923955


In [476]:
# Calculate USD price and USD marketcap
stock_data_filtered['previousCloseUSD'] = stock_data_filtered['previousClose'] / stock_data_filtered['USDExchangeRate']
stock_data_filtered['marketCapUSD'] = stock_data_filtered['marketCap'] / stock_data_filtered['USDExchangeRate']
stock_data_filtered.head(20)

Attribute,shortName,longName,country,currency,previousClose,impliedSharesOutstanding,marketCap,industry,sector,website,USDExchangeRate,previousCloseUSD,marketCapUSD
Ticker,,,,,,,,,,,,,
0973.HK,L'OCCITANE,L'Occitane International S.A.,Luxembourg,HKD,31.2,1472169984,46520573952,Household & Personal Products,Consumer Defensive,https://group.loccitane.com,7.823499,3.987985,5946261890.23607
1112.HK,H&H INTL HLDG,Health and Happiness (H&H) International Holdi...,Hong Kong,HKD,11.14,645561024,7088259584,Packaged Foods,Consumer Defensive,https://www.hh.global,7.823499,1.423915,906021664.219552
1346.HK,LEVER STYLE,Lever Style Corporation,Hong Kong,HKD,0.95,645899008,607145024,Apparel Manufacturing,Consumer Cyclical,https://www.leverstyle.com,7.823499,0.121429,77605304.73641
1H2.SI,$ Olive Tree,Olive Tree Estates Limited,Singapore,SGD,0.078,124756000,9035910,Real Estate - Diversified,Real Estate,https://www.olivetreeestates.com,1.348085,0.05786,6702774.676671
2897.TW,O-BANK CO LTD,"O-Bank Co., Ltd.",Taiwan,TWD,9.9,2716470016,27164700672,Banks - Regional,Financial Services,https://www.o-bank.com,31.995498,0.309419,849016341.986613
4755.T,RAKUTEN GROUP INC,"Rakuten Group, Inc.",Japan,JPY,879.1,2142140032,1854664867840,Internet Retail,Consumer Cyclical,https://global.rakuten.com/corp,151.355498,5.80818,12253700013.196745
6088.T,SIGMAXYZ HOLDINGS INC,SIGMAXYZ Holdings Inc.,Japan,JPY,1716.0,42581400,73618423808,Specialty Business Services,Industrials,https://www.sigmaxyz.com,151.355498,11.337547,486394117.034321
ABB.XA,AUSSIEBAND FPO,Aussie Broadband Limited,Australia,AUD,3.6,NaN,NaN,NaN,NaN,https://www.aussiebroadband.com.au,1.532450,2.349179,NaN
ABI.BR,AB INBEV,Anheuser-Busch InBev SA/NV,Belgium,EUR,55.82,282105984,110587289600,Beverages - Brewers,Consumer Defensive,https://www.ab-inbev.com,0.923955,60.414198,119689042864.641678


## Filter Universe of Stocks by USD MarketCap

In [477]:
def filter_index(df, size=True, num=True, size_thresh = 1000000000, num_thresh = 20):
    temp = df.copy()
    if size == True:
        temp = temp[temp['marketCapUSD'] >= size_thresh]
    
    if num == True:
        temp.sort_values(by=['marketCapUSD'], ascending=False, inplace = True)
        temp = temp.iloc[0:num_thresh,:]
            
    return temp



In [478]:
stock_index_top20 = filter_index(stock_data_filtered, size = False, num_thresh=20)
stock_index_top30 = filter_index(stock_data_filtered, size = False, num_thresh=30)
stock_index_over1b = filter_index(stock_data_filtered, size = True, num=False, size_thresh=1000000000)

print(stock_index_top20.shape, stock_index_top30.shape, stock_index_over1b.shape )

(20, 13) (30, 13) (20, 13)


In [479]:
stock_index_top20['currency']

Ticker
PG          USD
NESN.SW     CHF
KO          USD
UL          USD
ABI.BR      EUR
DANOY       USD
CABK.MC     EUR
FFH.TO      CAD
KIK.F       EUR
4755.T      JPY
GPS         USD
0973.HK     HKD
NTCO3.SA    BRL
COUR        USD
BFT.WA      PLN
WRBY        USD
ARZZ3.SA    BRL
COCO        USD
SOMA3.SA    BRL
LMND        USD
Name: currency, dtype: object

### Set Index and Get Price History

In [480]:
stock_index = stock_index_top20.copy()
start_date = s
end_date = e

In [481]:
def get_price_history2 (df, start = start_date, end=end_date):
    tickers_list = df.index.tolist()
    tickers_data= {}
    first_object = yf.Ticker(tickers_list[0])
    
    prices = pd.DataFrame.from_dict(first_object.history(start=start, end=end, interval = "1d"))
    prices.drop(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
    prices.columns = [tickers_list[0]]

    
    for ticker in tickers_list[1:]:
        ticker_object = yf.Ticker(ticker)
        try:
            temp = pd.DataFrame.from_dict(ticker_object.history(start=start, end=end, interval = "1d"))
            temp.drop(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
            temp.columns = [ticker]
            prices = prices.merge(temp, how="outer", left_index=True, right_index=True)
        except:
            continue
    
    
    # Collapse date across international zones:
    prices.index = pd.to_datetime(prices.index).date
    prices = prices.groupby(prices.index).ffill() 
    prices = prices.groupby(prices.index).bfill()
    prices = prices.groupby(prices.index).last()
    
    
    return prices

In [515]:
price_history_df = get_price_history2 (stock_index)
stock_universe = list(price_history_df.columns)
stock_universe

['PG',
 'NESN.SW',
 'KO',
 'UL',
 'ABI.BR',
 'DANOY',
 'CABK.MC',
 'FFH.TO',
 'KIK.F',
 '4755.T',
 'GPS',
 '0973.HK',
 'NTCO3.SA',
 'COUR',
 'BFT.WA',
 'WRBY',
 'ARZZ3.SA',
 'COCO',
 'SOMA3.SA',
 'LMND']

In [483]:
display(price_history_df.head())

,PG,NESN.SW,KO,UL,ABI.BR,DANOY,CABK.MC,FFH.TO,KIK.F,4755.T,GPS,0973.HK,NTCO3.SA,COUR,BFT.WA,WRBY,ARZZ3.SA,COCO,SOMA3.SA,LMND
2023-10-31,NaN,99.379997,NaN,NaN,53.380001,NaN,3.795,NaN,54.5,551.099976,NaN,20.049999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-01,148.671066,100.519997,55.549816,46.348526,54.389999,12.13,3.784,1142.130615,58.5,554.400024,12.837246,19.920000,11.632296,17.830000,1755.0,12.41,57.044178,28.040001,5.41,11.00
2023-11-02,150.489594,99.750000,56.189564,46.800129,55.110001,12.18,3.786,1137.299194,60.0,NaN,13.432940,20.000000,NaN,18.010000,1705.0,12.93,NaN,28.480000,NaN,16.17
2023-11-03,149.128189,NaN,55.845081,46.770679,NaN,12.24,NaN,1203.916626,NaN,NaN,13.939282,NaN,12.362782,18.049999,NaN,13.66,59.586674,28.930000,5.88,17.32
2023-11-05,NaN,99.870003,NaN,NaN,54.840000,NaN,3.818,NaN,60.5,568.799988,NaN,20.049999,NaN,NaN,1750.0,NaN,NaN,NaN,NaN,NaN


In [484]:
def get_country_info(price_df, info_df):
    tickers_list = price_df.columns
    
    #access currency, impliedSharesOutstanding (local currency): ## HAVE TO REVIEW FOR STOCK CHANGES
    currencies = []
    implied_SO = []
    for col in price_df.columns:
        idx = np.where(info_df.index == col)
        currencies.append(info_df.iloc[idx]['currency'].values[0])
        implied_SO.append(info_df.iloc[idx]['impliedSharesOutstanding'].values[0])
        
    for i in range(len(price_df.columns)):
        col = price_df.columns[i]
        price_df[col+'_currency'] = currencies[i]
        price_df[col+'_SO'] = implied_SO[i]
    
    return price_df, currencies
     

In [485]:
price_and_info_df, index_currencies = get_country_info(price_history_df, stock_index_top20)

In [486]:
price_and_info_df.head()

,PG,NESN.SW,KO,UL,ABI.BR,DANOY,CABK.MC,FFH.TO,KIK.F,4755.T,...,WRBY_currency,WRBY_SO,ARZZ3.SA_currency,ARZZ3.SA_SO,COCO_currency,COCO_SO,SOMA3.SA_currency,SOMA3.SA_SO,LMND_currency,LMND_SO
2023-10-31,NaN,99.379997,NaN,NaN,53.380001,NaN,3.795,NaN,54.5,551.099976,...,USD,118163000,BRL,110958000,USD,56929300,BRL,784747008,USD,70224400
2023-11-01,148.671066,100.519997,55.549816,46.348526,54.389999,12.13,3.784,1142.130615,58.5,554.400024,...,USD,118163000,BRL,110958000,USD,56929300,BRL,784747008,USD,70224400
2023-11-02,150.489594,99.750000,56.189564,46.800129,55.110001,12.18,3.786,1137.299194,60.0,NaN,...,USD,118163000,BRL,110958000,USD,56929300,BRL,784747008,USD,70224400
2023-11-03,149.128189,NaN,55.845081,46.770679,NaN,12.24,NaN,1203.916626,NaN,NaN,...,USD,118163000,BRL,110958000,USD,56929300,BRL,784747008,USD,70224400
2023-11-05,NaN,99.870003,NaN,NaN,54.840000,NaN,3.818,NaN,60.5,568.799988,...,USD,118163000,BRL,110958000,USD,56929300,BRL,784747008,USD,70224400


### Get historical exchange rates

In [505]:
def get_historical_exchange (price_df, currencies):
    #currencies = list(set(currencies))
    
    querystring = {"base":"USD"}

    #headers = {"X-RapidAPI-Key": "af4b092441msh3f4304a0bce0366p1680a3jsn93d4c3a0659f",
    headers = {"X-RapidAPI-Key": "884638c782msh3dc9c5784730cc0p1a0f42jsnf54c2327e84c",
        "X-RapidAPI-Host": "currency-conversion-and-exchange-rates.p.rapidapi.com"
              }
    daily_rate_df = pd.DataFrame()
    
    for idx in price_df.index:
        loc = np.where(price_df.index == idx)
        url = "https://currency-conversion-and-exchange-rates.p.rapidapi.com/" + str(idx)
        response = requests.get(url, headers=headers, params=querystring)
        response = response.json()
        print("Response is", response)
        
        daily_df = pd.DataFrame(response)
        daily_df = daily_df[['date', 'rates']].reset_index()
        daily_df = daily_df.pivot(columns='index', index='date', values='rates')

        daily_rate_df = pd.concat([daily_rate_df, daily_df], axis=0)
        display(daily_rate_df)
        
    daily_rate_df.index = pd.to_datetime(daily_rate_df.index).date
    return daily_rate_df


In [496]:
import warnings
warnings.filterwarnings("ignore")
daily_rates_df = get_historical_exchange(price_and_info_df, index_currencies)

Response is {'timestamp': 1698796799, 'base': 'USD', 'success': True, 'rates': {'ANG': 1.801847, 'TND': 3.175502, 'CAD': 1.387715, 'XCD': 2.70255, 'MVR': 15.380119, 'HRK': 7.168957, 'AUD': 1.577801, 'MWK': 1142.498827, 'XAG': 0.043781, 'MAD': 10.283019, 'PHP': 56.810192, 'NAD': 18.840275, 'GNF': 8650.000223, 'KES': 150.604886, 'MZN': 63.24995, 'BTN': 83.226354, 'MGA': 4519.999935, 'AZN': 1.699005, 'XAU': 0.000504, 'RON': 4.695202, 'CHF': 0.90988, 'EGP': 30.923797, 'BSD': 0.999808, 'TWD': 32.4179, 'GGP': 0.822657, 'LVL': 0.60489, 'MMK': 2099.539784, 'WST': 2.807677, 'ILS': 4.04708, 'BHD': 0.377032, 'GBP': 0.822985, 'TZS': 2499.999636, 'SDG': 600.502513, 'LAK': 20740.000263, 'DJF': 177.719533, 'BYN': 3.293379, 'LBP': 15032.499391, 'RWF': 1229.5, 'PEN': 3.820502, 'EUR': 0.9453, 'ZMK': 9001.20116, 'RSD': 110.783015, 'INR': 83.27745, 'MUR': 44.244275, 'BWP': 13.630073, 'GEL': 2.704994, 'KMF': 464.050201, 'UZS': 12237.49886, 'RUB': 93.324982, 'CUC': 1, 'BGN': 1.849019, 'JOD': 0.709303, 'NGN'

index,AED,AFN,ALL,AMD,ANG,AOA,ARS,AUD,AWG,AZN,...,XAU,XCD,XDR,XOF,XPF,YER,ZAR,ZMK,ZMW,ZWL
date,,,,,,,,,,,,,,,,,,,,,
2023-10-31,3.673015,74.999721,99.375013,402.141125,1.801847,831.000309,349.976698,1.577801,1.8,1.699005,...,0.000504,2.70255,0.762984,620.994978,112.924987,250.225009,18.639601,9001.20116,22.020182,321.999592


Response is {'timestamp': 1698883199, 'base': 'USD', 'success': True, 'rates': {'ANG': 1.801205, 'TND': 3.184502, 'CAD': 1.383675, 'XCD': 2.70255, 'MVR': 15.380299, 'HRK': 7.168957, 'AUD': 1.557475, 'MWK': 1144.493369, 'XAG': 0.043407, 'MAD': 10.30875, 'PHP': 56.679034, 'NAD': 18.425024, 'GNF': 8655.00006, 'KES': 150.849647, 'MZN': 63.24995, 'BTN': 83.284986, 'MGA': 4520.999956, 'AZN': 1.699529, 'XAU': 0.000504, 'RON': 4.695046, 'CHF': 0.905805, 'EGP': 30.899898, 'BSD': 0.999502, 'TWD': 32.378498, 'GGP': 0.82419, 'LVL': 0.60489, 'MMK': 2098.926362, 'WST': 2.766713, 'ILS': 4.01415, 'BHD': 0.377136, 'GBP': 0.821112, 'TZS': 2498.000492, 'SDG': 599.502307, 'LAK': 20741.999819, 'DJF': 177.719613, 'BYN': 3.292963, 'LBP': 15032.495873, 'RWF': 1231.5, 'PEN': 3.820499, 'EUR': 0.944745, 'ZMK': 9001.19652, 'RSD': 110.716032, 'INR': 85.201501, 'MUR': 44.25318, 'BWP': 13.644917, 'GEL': 2.714991, 'KMF': 467.249914, 'UZS': 12250.000237, 'RUB': 92.610099, 'CUC': 1, 'BGN': 1.848935, 'JOD': 0.709396, 'N

index,AED,AFN,ALL,AMD,ANG,AOA,ARS,AUD,AWG,AZN,...,XAU,XCD,XDR,XOF,XPF,YER,ZAR,ZMK,ZMW,ZWL
date,,,,,,,,,,,,,,,,,,,,,
2023-10-31,3.673015,74.999721,99.375013,402.141125,1.801847,831.000309,349.976698,1.577801,1.8000,1.699005,...,0.000504,2.70255,0.762984,620.994978,112.924987,250.225009,18.639601,9001.20116,22.020182,321.999592
2023-11-01,3.672991,73.417458,99.929725,402.390285,1.801205,831.488724,349.970103,1.557475,1.8025,1.699529,...,0.000504,2.70255,0.762806,616.496448,113.249863,250.301076,18.479750,9001.19652,22.087716,321.999592


Response is {'timestamp': 1698969599, 'base': 'USD', 'success': True, 'rates': {'ANG': 1.799184, 'TND': 3.184501, 'CAD': 1.374755, 'XCD': 2.70255, 'MVR': 15.380127, 'HRK': 7.168957, 'AUD': 1.555827, 'MWK': 1123.956236, 'XAG': 0.044029, 'MAD': 10.248731, 'PHP': 56.59504, 'NAD': 18.425033, 'GNF': 8574.79476, 'KES': 150.84973, 'MZN': 63.24995, 'BTN': 83.099575, 'MGA': 4521.837084, 'AZN': 1.697232, 'XAU': 0.000504, 'RON': 4.680301, 'CHF': 0.906165, 'EGP': 30.900985, 'BSD': 0.998329, 'TWD': 32.19597, 'GGP': 0.824698, 'LVL': 0.60489, 'MMK': 2096.502387, 'WST': 2.806618, 'ILS': 3.981885, 'BHD': 0.376984, 'GBP': 0.820335, 'TZS': 2498.000372, 'SDG': 599.504639, 'LAK': 20706.390369, 'DJF': 177.741269, 'BYN': 3.288914, 'LBP': 15003.168196, 'RWF': 1234.749872, 'PEN': 3.813663, 'EUR': 0.94185, 'ZMK': 9001.197998, 'RSD': 110.391969, 'INR': 83.24755, 'MUR': 44.243537, 'BWP': 13.508071, 'GEL': 2.715039, 'KMF': 467.250058, 'UZS': 12219.129472, 'RUB': 93.275056, 'CUC': 1, 'BGN': 1.842402, 'JOD': 0.70930

...

index,AED,AFN,ALL,AMD,ANG,AOA,ARS,AUD,AWG,AZN,...,XCD,XDR,XOF,XPF,YER,ZAR,ZMK,ZMW,ZWL,MRU
date,,,,,,,,,,,,,,,,,,,,,
2023-10-31,3.673015,74.999721,99.375013,402.141125,1.801847,831.000309,349.976698,1.577801,1.8000,1.699005,...,2.70255,0.762984,620.994978,112.924987,250.225009,18.639601,9001.201160,22.020182,321.999592,NaN
2023-11-01,3.672991,73.417458,99.929725,402.390285,1.801205,831.488724,349.970103,1.557475,1.8025,1.699529,...,2.70255,0.762806,616.496448,113.249863,250.301076,18.479750,9001.196520,22.087716,321.999592,NaN
2023-11-02,3.672996,73.628329,98.943445,401.687859,1.799184,831.514208,350.048395,1.555827,1.8025,1.697232,...,2.70255,0.761912,615.797750,113.249722,250.298711,18.417100,9001.197998,22.085377,321.999592,NaN
2023-11-03,3.673042,72.855147,97.904052,399.068468,1.787517,832.503981,347.193852,1.535650,1.8000,1.703970,...,2.70255,0.756963,611.045179,113.250364,250.350364,18.211910,9001.203591,22.042850,321.999592,NaN
2023-11-05,3.673020,72.855147,97.904052,399.068468,1.787517,832.492050,347.273108,1.534695,1.8000,1.697406,...,2.70255,0.756963,611.045179,113.250271,250.349729,18.268795,9001.198831,22.042850,321.999592,NaN
2023-11-06,3.673015,74.497874,97.649710,402.245655,1.802428,831.499594,350.023324,1.541040,1.8000,1.695940,...,2.70255,0.760709,610.508457,111.749917,250.349876,18.305100,9001.187145,22.171992,321.999592,NaN
2023-11-07,3.673024,73.495895,97.650122,402.659974,1.800804,827.070066,349.865188,1.554995,1.8000,1.703770,...,2.70255,0.760032,610.500193,111.750007,250.350141,18.348850,9001.197406,22.458131,321.999592,NaN
2023-11-08,3.673047,74.498342,97.849676,402.886463,1.803383,830.999753,349.975040,1.561134,1.8025,1.697851,...,2.70255,0.761178,613.496736,111.941272,250.349854,18.496899,9001.200733,22.538775,321.999592,NaN
2023-11-09,3.673009,74.498252,98.000332,402.680368,1.802150,831.496859,349.970102,1.571845,1.8000,1.696448,...,2.70255,0.760588,612.501962,112.349631,250.301441,18.668205,9001.202650,22.749046,321.999592,NaN


Response is {'timestamp': 1703807999, 'base': 'USD', 'success': True, 'rates': {'ANG': 1.811468, 'TND': 3.06425, 'CAD': 1.32311, 'XCD': 2.70255, 'MVR': 15.401194, 'HRK': 6.98177, 'AUD': 1.464998, 'MWK': 1667.502819, 'XAG': 0.041768, 'MAD': 9.888021, 'PHP': 55.373034, 'NAD': 18.53028, 'GNF': 8559.999751, 'KES': 156.999835, 'MZN': 63.24995, 'BTN': 83.617152, 'MGA': 4585.000231, 'AZN': 1.698678, 'XAU': 0.000484, 'RON': 4.490977, 'CHF': 0.84359, 'EGP': 30.920299, 'BSD': 1.005116, 'TWD': 30.713032, 'GGP': 0.781538, 'LVL': 0.60489, 'MMK': 2110.743533, 'WST': 2.700713, 'ILS': 3.627029, 'BHD': 0.376946, 'GBP': 0.78535, 'TZS': 2518.413985, 'SDG': 600.999884, 'LAK': 20549.999761, 'DJF': 178.959452, 'BYN': 3.31848, 'LBP': 15086.086131, 'RWF': 1262.5, 'PEN': 3.704505, 'EUR': 0.90354, 'ZMK': 9001.192558, 'RSD': 105.876044, 'INR': 82.891022, 'MUR': 43.904202, 'BWP': 13.482293, 'GEL': 2.685008, 'KMF': 445.149925, 'UZS': 12345.000454, 'RUB': 88.474973, 'CUC': 1, 'BGN': 1.767845, 'JOD': 0.709598, 'NGN'

index,AED,AFN,ALL,AMD,ANG,AOA,ARS,AUD,AWG,AZN,...,XCD,XDR,XOF,XPF,YER,ZAR,ZMK,ZMW,ZWL,MRU
date,,,,,,,,,,,,,,,,,,,,,
2023-10-31,3.673015,74.999721,99.375013,402.141125,1.801847,831.000309,349.976698,1.577801,1.8000,1.699005,...,2.70255,0.762984,620.994978,112.924987,250.225009,18.639601,9001.201160,22.020182,321.999592,NaN
2023-11-01,3.672991,73.417458,99.929725,402.390285,1.801205,831.488724,349.970103,1.557475,1.8025,1.699529,...,2.70255,0.762806,616.496448,113.249863,250.301076,18.479750,9001.196520,22.087716,321.999592,NaN
2023-11-02,3.672996,73.628329,98.943445,401.687859,1.799184,831.514208,350.048395,1.555827,1.8025,1.697232,...,2.70255,0.761912,615.797750,113.249722,250.298711,18.417100,9001.197998,22.085377,321.999592,NaN
2023-11-03,3.673042,72.855147,97.904052,399.068468,1.787517,832.503981,347.193852,1.535650,1.8000,1.703970,...,2.70255,0.756963,611.045179,113.250364,250.350364,18.211910,9001.203591,22.042850,321.999592,NaN
2023-11-05,3.673020,72.855147,97.904052,399.068468,1.787517,832.492050,347.273108,1.534695,1.8000,1.697406,...,2.70255,0.756963,611.045179,113.250271,250.349729,18.268795,9001.198831,22.042850,321.999592,NaN
2023-11-06,3.673015,74.497874,97.649710,402.245655,1.802428,831.499594,350.023324,1.541040,1.8000,1.695940,...,2.70255,0.760709,610.508457,111.749917,250.349876,18.305100,9001.187145,22.171992,321.999592,NaN
2023-11-07,3.673024,73.495895,97.650122,402.659974,1.800804,827.070066,349.865188,1.554995,1.8000,1.703770,...,2.70255,0.760032,610.500193,111.750007,250.350141,18.348850,9001.197406,22.458131,321.999592,NaN
2023-11-08,3.673047,74.498342,97.849676,402.886463,1.803383,830.999753,349.975040,1.561134,1.8025,1.697851,...,2.70255,0.761178,613.496736,111.941272,250.349854,18.496899,9001.200733,22.538775,321.999592,NaN
2023-11-09,3.673009,74.498252,98.000332,402.680368,1.802150,831.496859,349.970102,1.571845,1.8000,1.696448,...,2.70255,0.760588,612.501962,112.349631,250.301441,18.668205,9001.202650,22.749046,321.999592,NaN


Response is {'timestamp': 1703894399, 'base': 'USD', 'success': True, 'rates': {'ANG': 1.802398, 'TND': 3.06425, 'CAD': 1.32635, 'XCD': 2.70255, 'MVR': 15.403741, 'HRK': 6.98177, 'AUD': 1.467782, 'MWK': 1683.459733, 'XAG': 0.042027, 'MAD': 9.898878, 'PHP': 55.403706, 'NAD': 18.530377, 'GNF': 8599.005029, 'KES': 157.010385, 'MZN': 63.24995, 'BTN': 83.190683, 'MGA': 4608.161791, 'AZN': 1.70397, 'XAU': 0.000485, 'RON': 4.506604, 'CHF': 0.841104, 'EGP': 30.859753, 'BSD': 1.000117, 'TWD': 30.68355, 'GGP': 0.784894, 'LVL': 0.60489, 'MMK': 2100.183284, 'WST': 2.69967, 'ILS': 3.60085, 'BHD': 0.376935, 'GBP': 0.784929, 'TZS': 2510.000336, 'SDG': 601.00034, 'LAK': 20538.100397, 'DJF': 177.720394, 'BYN': 3.301823, 'LBP': 15030.697003, 'RWF': 1259.062633, 'PEN': 3.693006, 'EUR': 0.904604, 'ZMK': 9001.203591, 'RSD': 106.222038, 'INR': 83.24885, 'MUR': 44.020379, 'BWP': 13.405925, 'GEL': 2.68504, 'KMF': 445.150384, 'UZS': 12339.626389, 'RUB': 89.250373, 'CUC': 1, 'BGN': 1.771495, 'JOD': 0.709604, 'N

index,AED,AFN,ALL,AMD,ANG,AOA,ARS,AUD,AWG,AZN,...,XCD,XDR,XOF,XPF,YER,ZAR,ZMK,ZMW,ZWL,MRU
date,,,,,,,,,,,,,,,,,,,,,
2023-10-31,3.673015,74.999721,99.375013,402.141125,1.801847,831.000309,349.976698,1.577801,1.8000,1.699005,...,2.70255,0.762984,620.994978,112.924987,250.225009,18.639601,9001.201160,22.020182,321.999592,NaN
2023-11-01,3.672991,73.417458,99.929725,402.390285,1.801205,831.488724,349.970103,1.557475,1.8025,1.699529,...,2.70255,0.762806,616.496448,113.249863,250.301076,18.479750,9001.196520,22.087716,321.999592,NaN
2023-11-02,3.672996,73.628329,98.943445,401.687859,1.799184,831.514208,350.048395,1.555827,1.8025,1.697232,...,2.70255,0.761912,615.797750,113.249722,250.298711,18.417100,9001.197998,22.085377,321.999592,NaN
2023-11-03,3.673042,72.855147,97.904052,399.068468,1.787517,832.503981,347.193852,1.535650,1.8000,1.703970,...,2.70255,0.756963,611.045179,113.250364,250.350364,18.211910,9001.203591,22.042850,321.999592,NaN
2023-11-05,3.673020,72.855147,97.904052,399.068468,1.787517,832.492050,347.273108,1.534695,1.8000,1.697406,...,2.70255,0.756963,611.045179,113.250271,250.349729,18.268795,9001.198831,22.042850,321.999592,NaN
2023-11-06,3.673015,74.497874,97.649710,402.245655,1.802428,831.499594,350.023324,1.541040,1.8000,1.695940,...,2.70255,0.760709,610.508457,111.749917,250.349876,18.305100,9001.187145,22.171992,321.999592,NaN
2023-11-07,3.673024,73.495895,97.650122,402.659974,1.800804,827.070066,349.865188,1.554995,1.8000,1.703770,...,2.70255,0.760032,610.500193,111.750007,250.350141,18.348850,9001.197406,22.458131,321.999592,NaN
2023-11-08,3.673047,74.498342,97.849676,402.886463,1.803383,830.999753,349.975040,1.561134,1.8025,1.697851,...,2.70255,0.761178,613.496736,111.941272,250.349854,18.496899,9001.200733,22.538775,321.999592,NaN
2023-11-09,3.673009,74.498252,98.000332,402.680368,1.802150,831.496859,349.970102,1.571845,1.8000,1.696448,...,2.70255,0.760588,612.501962,112.349631,250.301441,18.668205,9001.202650,22.749046,321.999592,NaN


In [497]:
daily_rates_df.head()

index,AED,AFN,ALL,AMD,ANG,AOA,ARS,AUD,AWG,AZN,...,XCD,XDR,XOF,XPF,YER,ZAR,ZMK,ZMW,ZWL,MRU
date,,,,,,,,,,,,,,,,,,,,,
2023-10-31,3.673015,74.999721,99.375013,402.141125,1.801847,831.000309,349.976698,1.577801,1.8000,1.699005,...,2.70255,0.762984,620.994978,112.924987,250.225009,18.639601,9001.201160,22.020182,321.999592,NaN
2023-11-01,3.672991,73.417458,99.929725,402.390285,1.801205,831.488724,349.970103,1.557475,1.8025,1.699529,...,2.70255,0.762806,616.496448,113.249863,250.301076,18.479750,9001.196520,22.087716,321.999592,NaN
2023-11-02,3.672996,73.628329,98.943445,401.687859,1.799184,831.514208,350.048395,1.555827,1.8025,1.697232,...,2.70255,0.761912,615.797750,113.249722,250.298711,18.417100,9001.197998,22.085377,321.999592,NaN
2023-11-03,3.673042,72.855147,97.904052,399.068468,1.787517,832.503981,347.193852,1.535650,1.8000,1.703970,...,2.70255,0.756963,611.045179,113.250364,250.350364,18.211910,9001.203591,22.042850,321.999592,NaN
2023-11-05,3.673020,72.855147,97.904052,399.068468,1.787517,832.492050,347.273108,1.534695,1.8000,1.697406,...,2.70255,0.756963,611.045179,113.250271,250.349729,18.268795,9001.198831,22.042850,321.999592,NaN


In [504]:
daily_price_and_info_df = price_and_info_df.join(daily_rates_df)
display(daily_price_and_info_df.head())

,PG,NESN.SW,KO,UL,ABI.BR,DANOY,CABK.MC,FFH.TO,KIK.F,4755.T,...,XCD,XDR,XOF,XPF,YER,ZAR,ZMK,ZMW,ZWL,MRU
2023-10-31,NaN,99.379997,NaN,NaN,53.380001,NaN,3.795,NaN,54.5,551.099976,...,2.70255,0.762984,620.994978,112.924987,250.225009,18.639601,9001.201160,22.020182,321.999592,NaN
2023-11-01,148.671066,100.519997,55.549816,46.348526,54.389999,12.13,3.784,1142.130615,58.5,554.400024,...,2.70255,0.762806,616.496448,113.249863,250.301076,18.479750,9001.196520,22.087716,321.999592,NaN
2023-11-02,150.489594,99.750000,56.189564,46.800129,55.110001,12.18,3.786,1137.299194,60.0,NaN,...,2.70255,0.761912,615.797750,113.249722,250.298711,18.417100,9001.197998,22.085377,321.999592,NaN
2023-11-03,149.128189,NaN,55.845081,46.770679,NaN,12.24,NaN,1203.916626,NaN,NaN,...,2.70255,0.756963,611.045179,113.250364,250.350364,18.211910,9001.203591,22.042850,321.999592,NaN
2023-11-05,NaN,99.870003,NaN,NaN,54.840000,NaN,3.818,NaN,60.5,568.799988,...,2.70255,0.756963,611.045179,113.250271,250.349729,18.268795,9001.198831,22.042850,321.999592,NaN


In [507]:
daily_price_and_info_df

,PG,NESN.SW,KO,UL,ABI.BR,DANOY,CABK.MC,FFH.TO,KIK.F,4755.T,...,XCD,XDR,XOF,XPF,YER,ZAR,ZMK,ZMW,ZWL,MRU
2023-10-31,NaN,99.379997,NaN,NaN,53.380001,NaN,3.795,NaN,54.5,551.099976,...,2.70255,0.762984,620.994978,112.924987,250.225009,18.639601,9001.201160,22.020182,321.999592,NaN
2023-11-01,148.671066,100.519997,55.549816,46.348526,54.389999,12.13,3.784,1142.130615,58.5,554.400024,...,2.70255,0.762806,616.496448,113.249863,250.301076,18.479750,9001.196520,22.087716,321.999592,NaN
2023-11-02,150.489594,99.750000,56.189564,46.800129,55.110001,12.18,3.786,1137.299194,60.0,NaN,...,2.70255,0.761912,615.797750,113.249722,250.298711,18.417100,9001.197998,22.085377,321.999592,NaN
2023-11-03,149.128189,NaN,55.845081,46.770679,NaN,12.24,NaN,1203.916626,NaN,NaN,...,2.70255,0.756963,611.045179,113.250364,250.350364,18.211910,9001.203591,22.042850,321.999592,NaN
2023-11-05,NaN,99.870003,NaN,NaN,54.840000,NaN,3.818,NaN,60.5,568.799988,...,2.70255,0.756963,611.045179,113.250271,250.349729,18.268795,9001.198831,22.042850,321.999592,NaN
2023-11-06,149.992722,99.620003,56.071457,46.859035,55.360001,12.21,3.765,1233.279541,58.5,563.099976,...,2.70255,0.760709,610.508457,111.749917,250.349876,18.305100,9001.187145,22.171992,321.999592,NaN
2023-11-07,149.644913,99.070000,56.278141,46.927757,55.660000,12.18,3.793,1205.638672,59.5,548.599976,...,2.70255,0.760032,610.500193,111.750007,250.350141,18.348850,9001.197406,22.458131,321.999592,NaN
2023-11-08,149.068558,99.910004,56.189564,46.937576,56.279999,12.21,3.866,1227.641113,60.5,550.599976,...,2.70255,0.761178,613.496736,111.941272,250.349854,18.496899,9001.200733,22.538775,321.999592,NaN
2023-11-09,149.406433,99.379997,55.766346,47.654255,55.410000,12.23,3.856,1215.252441,60.0,554.000000,...,2.70255,0.760588,612.501962,112.349631,250.301441,18.668205,9001.202650,22.749046,321.999592,NaN
2023-11-10,150.459778,NaN,55.825401,47.556076,NaN,12.29,NaN,1222.898193,NaN,NaN,...,2.70255,0.760540,612.503600,112.350364,250.303595,18.725180,9001.203591,22.819741,321.999592,NaN


In [543]:
def convert_price_mktcap_USD (px_info_df, stock_list):
    
    for stock in stock_list:
        col_name = stock+"_currency"
        col2_name = stock+"_SO"
        curr = px_info_df[col_name][0]
        shares_out = px_info_df[col2_name][0]
        new_col = stock + "_px_USD"
        new_col2 = stock + "_MktCap_USD"
        px_info_df[new_col] = px_info_df[stock] / px_info_df[curr]
        px_info_df[new_col2] = px_info_df[new_col] * shares_out
    
    return px_info_df
    

In [547]:
daily_price_MC_usd_all = convert_price_mktcap_USD(daily_price_and_info_df, stock_universe)

In [546]:
#daily_price_MC_usd.to_csv('price_MC_USD_all.csv')

### Refine DF and calculate PW and MW indicies:

In [558]:
def get_df_in_USD(df_all):
    cols_to_keep = []
    for col in df_all:
        if "USD" in col:
            cols_to_keep.append(col)
    cols_to_keep.remove('USD')
    df_USD = df_all[cols_to_keep]
    
    px = []
    MC = []
    
    for col in cols_to_keep:
        if "_px" in col:
            px.append(col)
        if "_MktCap" in col:
            MC.append(col)

    df_USD_px = df_USD[px]
    df_MC_px = df_USD[MC]
    
    return df_USD, df_USD_px, df_MC_px

In [559]:
daily_info_usd, daily_px_USD, daily_MC_USD = get_df_in_USD(daily_price_MC_usd_all)

,PG_px_USD,NESN.SW_px_USD,KO_px_USD,UL_px_USD,ABI.BR_px_USD,DANOY_px_USD,CABK.MC_px_USD,FFH.TO_px_USD,KIK.F_px_USD,4755.T_px_USD,...,GPS_MktCap_USD,0973.HK_MktCap_USD,NTCO3.SA_MktCap_USD,COUR_MktCap_USD,BFT.WA_MktCap_USD,WRBY_MktCap_USD,ARZZ3.SA_MktCap_USD,COCO_MktCap_USD,SOMA3.SA_MktCap_USD,LMND_MktCap_USD
2023-10-31,NaN,109.223191,NaN,NaN,56.468847,NaN,4.014599,NaN,57.653655,3.642420,...,NaN,3.772385e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-01,148.671066,110.973109,55.549816,46.348526,57.571090,12.13,4.005314,825.432717,61.921471,3.681092,...,4.827741e+09,3.748953e+09,3.241538e+09,2.798347e+09,1.233514e+09,1.466403e+09,1.277348e+09,1.596298e+09,8.567736e+08,7.724684e+08
2023-11-02,150.489594,110.079290,56.189564,46.800129,58.512503,12.18,4.019748,827.274092,63.704412,NaN,...,5.051766e+09,3.762749e+09,NaN,2.826597e+09,1.207527e+09,1.527848e+09,NaN,1.621346e+09,NaN,1.135529e+09
2023-11-03,149.128189,NaN,55.845081,46.770679,NaN,12.24,NaN,880.603172,NaN,NaN,...,5.242188e+09,NaN,3.483123e+09,2.832875e+09,NaN,1.614107e+09,1.349006e+09,1.646965e+09,9.414843e+08,1.216287e+09
2023-11-05,NaN,111.070335,NaN,NaN,58.854463,NaN,4.097490,NaN,64.928793,3.805420,...,NaN,3.772431e+09,NaN,NaN,1.250677e+09,NaN,NaN,NaN,NaN,NaN


,PG_px_USD,NESN.SW_px_USD,KO_px_USD,UL_px_USD,ABI.BR_px_USD,DANOY_px_USD,CABK.MC_px_USD,FFH.TO_px_USD,KIK.F_px_USD,4755.T_px_USD,GPS_px_USD,0973.HK_px_USD,NTCO3.SA_px_USD,COUR_px_USD,BFT.WA_px_USD,WRBY_px_USD,ARZZ3.SA_px_USD,COCO_px_USD,SOMA3.SA_px_USD,LMND_px_USD
2023-10-31,NaN,109.223191,NaN,NaN,56.468847,NaN,4.014599,NaN,57.653655,3.642420,NaN,2.562466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-01,148.671066,110.973109,55.549816,46.348526,57.571090,12.13,4.005314,825.432717,61.921471,3.681092,12.837246,2.546549,2.347495,17.830000,415.744370,12.41,11.511992,28.040001,1.091783,11.00
2023-11-02,150.489594,110.079290,56.189564,46.800129,58.512503,12.18,4.019748,827.274092,63.704412,NaN,13.432940,2.555920,NaN,18.010000,406.985737,12.93,NaN,28.480000,NaN,16.17
2023-11-03,149.128189,NaN,55.845081,46.770679,NaN,12.24,NaN,880.603172,NaN,NaN,13.939282,NaN,2.522448,18.049999,NaN,13.66,12.157806,28.930000,1.199730,17.32
2023-11-05,NaN,111.070335,NaN,NaN,58.854463,NaN,4.097490,NaN,64.928793,3.805420,NaN,2.562497,NaN,NaN,421.529308,NaN,NaN,NaN,NaN,NaN


,PG_MktCap_USD,NESN.SW_MktCap_USD,KO_MktCap_USD,UL_MktCap_USD,ABI.BR_MktCap_USD,DANOY_MktCap_USD,CABK.MC_MktCap_USD,FFH.TO_MktCap_USD,KIK.F_MktCap_USD,4755.T_MktCap_USD,GPS_MktCap_USD,0973.HK_MktCap_USD,NTCO3.SA_MktCap_USD,COUR_MktCap_USD,BFT.WA_MktCap_USD,WRBY_MktCap_USD,ARZZ3.SA_MktCap_USD,COCO_MktCap_USD,SOMA3.SA_MktCap_USD,LMND_MktCap_USD
2023-10-31,NaN,2.863078e+11,NaN,NaN,1.593020e+10,NaN,2.959855e+10,NaN,1.101658e+10,7.802574e+09,NaN,3.772385e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-01,3.498260e+11,2.908949e+11,2.398424e+11,1.165072e+11,1.624115e+10,3.904598e+10,2.953009e+10,1.973519e+10,1.183208e+10,7.885414e+09,4.827741e+09,3.748953e+09,3.241538e+09,2.798347e+09,1.233514e+09,1.466403e+09,1.277348e+09,1.596298e+09,8.567736e+08,7.724684e+08
2023-11-02,3.541050e+11,2.885519e+11,2.426046e+11,1.176424e+11,1.650673e+10,3.920693e+10,2.963652e+10,1.977921e+10,1.217277e+10,NaN,5.051766e+09,3.762749e+09,NaN,2.826597e+09,1.207527e+09,1.527848e+09,NaN,1.621346e+09,NaN,1.135529e+09
2023-11-03,3.509016e+11,NaN,2.411173e+11,1.175684e+11,NaN,3.940007e+10,NaN,2.105425e+10,NaN,NaN,5.242188e+09,NaN,3.483123e+09,2.832875e+09,NaN,1.614107e+09,1.349006e+09,1.646965e+09,9.414843e+08,1.216287e+09
2023-11-05,NaN,2.911498e+11,NaN,NaN,1.660320e+10,NaN,3.020969e+10,NaN,1.240672e+10,8.151743e+09,NaN,3.772431e+09,NaN,NaN,1.250677e+09,NaN,NaN,NaN,NaN,NaN


In [572]:
def calc_px_weighted_index(df):
    # Fill in missing days:
    pw_index = df.ffill()
    n, m = pw_index.shape
    pw_index['Index'] = pw_index.iloc[:,0:m].sum(axis=1)
    

In [573]:
calc_px_weighted_index(daily_px_USD)

,PG_px_USD,NESN.SW_px_USD,KO_px_USD,UL_px_USD,ABI.BR_px_USD,DANOY_px_USD,CABK.MC_px_USD,FFH.TO_px_USD,KIK.F_px_USD,4755.T_px_USD,...,0973.HK_px_USD,NTCO3.SA_px_USD,COUR_px_USD,BFT.WA_px_USD,WRBY_px_USD,ARZZ3.SA_px_USD,COCO_px_USD,SOMA3.SA_px_USD,LMND_px_USD,Index
2023-10-31,NaN,109.223191,NaN,NaN,56.468847,NaN,4.014599,NaN,57.653655,3.642420,...,2.562466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,233.565177
2023-11-01,148.671066,110.973109,55.549816,46.348526,57.571090,12.13,4.005314,825.432717,61.921471,3.681092,...,2.546549,2.347495,17.830000,415.744370,12.41,11.511992,28.040001,1.091783,11.00,1841.643636
2023-11-02,150.489594,110.079290,56.189564,46.800129,58.512503,12.18,4.019748,827.274092,63.704412,3.681092,...,2.555920,2.347495,18.010000,406.985737,12.93,11.511992,28.480000,1.091783,16.17,1846.446291
2023-11-03,149.128189,110.079290,55.845081,46.770679,58.512503,12.24,4.019748,880.603172,63.704412,3.681092,...,2.555920,2.522448,18.049999,406.985737,13.66,12.157806,28.930000,1.199730,17.32,1901.905089
2023-11-05,149.128189,111.070335,55.845081,46.770679,58.854463,12.24,4.097490,880.603172,64.928793,3.805420,...,2.562497,2.522448,18.049999,421.529308,13.66,12.157806,28.930000,1.199730,17.32,1919.214694


In [135]:
def get_price_history2 (df, start, end):
    tickers_list = df['Ticker'].tolist()
    tickers_data= {}
    first_object = yf.Ticker(tickers_list[0])
    prices = pd.DataFrame.from_dict(first_object.history(start=start, end=end, interval = "1d"))
    prices.drop(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
    prices.columns = [tickers_list[0]]
    
    for ticker in tickers_list[1:]:
        ticker_object = yf.Ticker(ticker)
        try:
            temp = pd.DataFrame.from_dict(ticker_object.history(start=start, end=end, interval = "1d"))
            temp.drop(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
            temp.columns = [ticker]
            prices = prices.merge(temp, how="outer", left_index=True, right_index=True)
        except:
            continue

    return prices

In [137]:
prices = get_price_history2(stocks_updated, start=s, end=e)

KCT.AQ: Period '1mo' is invalid, must be one of ['1d', '5d']


In [138]:
print(prices.shape)

top20=prices.iloc[:,0:20]
print(top20.shape)
top30=prices.iloc[:,0:30]
print(top30.shape)

(4535, 67)
(4535, 20)
(4535, 30)


In [159]:
def calc_index2(df, num_in_index = 30, divisor = 1000000000):
    cols = df.columns
    mkt_cap_cols = []
    for col in cols:
        if "marketCapUSD" in col:
            mkt_cap_cols.append(col)
    
    mkt_cap_df = df[mkt_cap_cols].copy()
    
    mkt_cap_df = mkt_cap_df.iloc[0:num_in_index,:]
    #display(mkt_cap_df)
    
    # Sort by beginning mkt cap
    baseline_order = mkt_cap_df.iloc[0,:].sort_values(ascending=False)
    #baseline_all = baseline_order.index.to_list()

    
    mkt_cap_df['Total_Index'] = mkt_cap_df.sum(axis=0)/divisor
    
    return mkt_cap_df

In [160]:
calc_index2(stocks_updated)

,marketCapUSD,Total_Index
0,3.789540e+11,NaN
1,2.759200e+11,NaN
2,2.616030e+11,NaN
3,1.255920e+11,NaN
4,1.191740e+11,NaN
5,1.176780e+11,NaN
6,4.152268e+10,NaN
7,3.796921e+10,NaN
8,2.637475e+10,NaN
9,1.257604e+10,NaN


In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(3, 1, figsize = (20,18))
ax0.plot(normed_data0, color = 'seagreen', linewidth=2, label = 'With Subs')
ax0.legend(loc=3)
ax1.plot(normed_data1, color = 'royalblue', linewidth=2,label='Cert B Corps')
ax1.legend(loc=3)
ax2.plot(normed_data0, color = 'seagreen', linewidth=2, label = 'With Subs')
ax2.plot(normed_data1, color = 'royalblue', linewidth=2, label = 'Cert B Corps')
ax2.plot(normed_spx, color='darkorange', linewidth=2, label = 'SPX')
ax2.legend(loc=3)
plt.show()

In [105]:
## This is the old code.....

def get_price_history (df, start, end, add_subs = False):
    tickers_list = df['Ticker'].tolist()
    if add_subs:
        tickers_list = tickers_list + public_subs

    tickers_data= {}
    first_object = yf.Ticker(tickers_list[0])
    prices = pd.DataFrame.from_dict(first_object.history(start=start, end=end, interval = "1d"))
    prices.drop(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
    prices.columns = [tickers_list[0]]
    
    for ticker in tickers_list[1:]:
        ticker_object = yf.Ticker(ticker)
        temp = pd.DataFrame.from_dict(ticker_object.history(start=start, end=end, interval = "1d"))
        temp.drop(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
        temp.columns = [ticker]
        prices = prices.merge(temp, how="outer", left_index=True, right_index=True)

    return prices

In [104]:
#px_df = get_price_history(filtered, start=s, end = e)
spx_object = yf.Ticker("^GSPC")
spx = pd.DataFrame.from_dict(spx_object.history(start=s, end=e, interval = "1d"))
spx

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-01-03 00:00:00-05:00,4778.140137,4796.640137,4758.169922,4796.560059,3831020000,0.0,0.0
2022-01-04 00:00:00-05:00,4804.509766,4818.620117,4774.270020,4793.540039,4683170000,0.0,0.0
2022-01-05 00:00:00-05:00,4787.990234,4797.700195,4699.439941,4700.580078,4887960000,0.0,0.0
2022-01-06 00:00:00-05:00,4693.390137,4725.009766,4671.259766,4696.049805,4295280000,0.0,0.0
2022-01-07 00:00:00-05:00,4697.660156,4707.950195,4662.740234,4677.029785,4181510000,0.0,0.0
...,...,...,...,...,...,...,...
2024-02-15 00:00:00-05:00,5003.140137,5032.720215,4999.439941,5029.729980,4137970000,0.0,0.0
2024-02-16 00:00:00-05:00,5031.129883,5038.700195,4999.520020,5005.569824,3833270000,0.0,0.0
2024-02-20 00:00:00-05:00,4989.319824,4993.709961,4955.020020,4975.509766,4034880000,0.0,0.0


In [ ]:
def add_shares_out (df, add_subs = False):
    df['BIRD_SharesOut'] = 152850000
    df['BIRD_MktCap'] = df['BIRD_SharesOut'] * df['BIRD']

    df['AMAL_SharesOut'] = 30430000
    df['AMAL_MktCap'] = df['AMAL_SharesOut'] * df['AMAL']

    df['COUR_SharesOut'] = 156950000
    df['COUR_MktCap'] = df['COUR_SharesOut'] * df['COUR']

    df['GROV_SharesOut'] = 37450000
    df['GROV_MktCap'] = df['GROV_SharesOut'] * df['GROV']

    df['LMND_SharesOut'] = 69920000
    df['LMND_MktCap'] = df['LMND_SharesOut'] * df['LMND']

    df['COCO_SharesOut'] = 56800000
    df['COCO_MktCap'] = df['COCO_SharesOut'] * df['COCO']

    df['VITL_SharesOut'] = 41600000
    df['VITL_MktCap'] = df['VITL_SharesOut'] * df['VITL']

    df['VVPR_SharesOut'] = 3270000
    df['VVPR_MktCap'] = df['VVPR_SharesOut'] * df['VVPR']

    df['WRBY_SharesOut'] = 117640000
    df['WRBY_MktCap'] = df['WRBY_SharesOut'] * df['WRBY']

    df['ZVIA_SharesOut'] = 50410000
    df['ZVIA_MktCap'] = df['ZVIA_SharesOut'] * df['ZVIA']


    if add_subs:
        df['UL_SharesOut'] = 2499020000
        df['UL_MktCap'] = df['UL_SharesOut'] * df['UL']

        df['PG_SharesOut'] = 2353020000
        df['PG_MktCap'] = df['PG_SharesOut'] * df['PG']

        df['BUD_SharesOut'] = 1986770000
        df['BUD_MktCap'] = df['BUD_SharesOut'] * df['BUD']

        df['KO_SharesOut'] = 4312460000
        df['KO_MktCap'] = df['KO_SharesOut'] * df['KO']


        df['GPS_SharesOut'] = 370830000 
        df['GPS_MktCap'] = df['GPS_SharesOut'] * df['GPS']


        df['OPY_SharesOut'] = 10290000
        df['OPY_MktCap'] = df['OPY_SharesOut'] * df['OPY']


        df['HAIN_SharesOut'] = 89830000
        df['HAIN_MktCap'] = df['HAIN_SharesOut'] * df['HAIN']


        df['DANOY_SharesOut'] = 2499020000
        df['DANOY_MktCap'] = df['DANOY_SharesOut'] * df['DANOY']
        
    col_order = df.columns.sort_values().tolist()
    df = df[col_order]
    
    return df


In [ ]:
#col_order = px_df.columns.sort_values().tolist()
#px_df = px_df[col_order]


In [ ]:
#px_df.to_csv('Public_B_Corps.csv')

In [ ]:
## This is the old code


def calc_index(df, divisor = 1000000000):
    cols = df.columns
    mkt_cap_cols = []
    for col in cols:
        if "MktCap" in col:
            mkt_cap_cols.append(col)
    
    mkt_cap_df = df[mkt_cap_cols].copy()
    
    # Sort by beginning mkt cap
    baseline_order = mkt_cap_df.iloc[0,:].sort_values(ascending=False)
    baseline_all = baseline_order.index.to_list()
    
    mkt_cap_df['Total_Index'] = mkt_cap_df[baseline_all].sum(axis=1)/divisor
    
    return mkt_cap_df

In [ ]:
def separate_subs (df, show_chart = True):

    px0 = get_price_history(df, start="2022-01-01", end = "2024-02-23", add_subs = True)
    px0 = add_shares_out(px0, add_subs = True)
    #px0.to_csv('24_03_19_BCorp_Index.csv')
    mkt_cap0 = calc_index(px0, divisor = 100000000)
    index0 = mkt_cap0[['Total_Index']]
    normed_data0 = index0.copy()
    normed_data0 = normed_data0.div(normed_data0.iloc[0]).fillna(0)


    px1 = get_price_history(df, start="2022-01-01", end = "2024-02-23", add_subs = False)
    px1 = add_shares_out(px1, add_subs = False)
    mkt_cap1 = calc_index(px1, divisor = 10000000)
    index1 = mkt_cap1[['Total_Index']]
    normed_data1 = index1.copy()
    normed_data1 = normed_data1.div(normed_data1.iloc[0]).fillna(0)
    
    # Add SPX:
    normed_spx = spx['Close'].copy()
    normed_spx = normed_spx.div(normed_spx.iloc[0]).fillna(0)
    
    if show_chart:
        fig, (ax0, ax1, ax2) = plt.subplots(3, 1, figsize = (20,18))
        ax0.plot(normed_data0, color = 'seagreen', linewidth=2, label = 'With Subs')
        ax0.legend(loc=3)
        ax1.plot(normed_data1, color = 'royalblue', linewidth=2,label='Cert B Corps')
        ax1.legend(loc=3)
        ax2.plot(normed_data0, color = 'seagreen', linewidth=2, label = 'With Subs')
        ax2.plot(normed_data1, color = 'royalblue', linewidth=2, label = 'Cert B Corps')
        ax2.plot(normed_spx, color='darkorange', linewidth=2, label = 'SPX')
        ax2.legend(loc=3)
        plt.show()
        
    return px0, px1

In [ ]:
all_px, bcorp_px = separate_subs(us, show_chart = True)

In [ ]:
all_px.to_csv("B_Corp_Stocks.csv")

## Construct a Subset - look at reweighting?

In [ ]:
def determine_reweighting (df, n=8):
    cols = df.columns
    mkt_cap_cols = []
    for col in cols:
        if "MktCap" in col:
            mkt_cap_cols.append(col)
    mkt_cap_df = df[mkt_cap_cols].copy()
    
    # Sort by beginning mkt cap
    baseline_order = mkt_cap_df.iloc[0,:].sort_values(ascending=False)
    baseline_all = baseline_order.index.to_list()
    baseline_n = baseline_order.index[0:n].to_list()
    
    #Add Quarterly Flag
    mkt_cap_df['Quarter'] = mkt_cap_df.index.quarter
    
    mkt_cap_df['Reweight'] = mkt_cap_df['Quarter'] - mkt_cap_df['Quarter'].shift(periods=1)
    
    quarterly = mkt_cap_df[mkt_cap_df['Reweight'].isin([1,-3])]
    
    for i in range(quarterly.shape[0]):
        order = quarterly.iloc[i,:].sort_values(ascending=False)
        order_all = order.index.to_list()
        order_n = order.index[0:n].to_list()
        for j in order_n:
            if j not in baseline_n:
                print("Need to Reweight!", i, j)
                print("Baseline n", baseline_n)
                print("New n", order_n)
        baseline_n = order_n
    
    return quarterly
    #mkt_cap_df['Total_Index'] = mkt_cap_df[baseline_all].sum(axis=1)/10000000
    #mkt_cap_df['Total_Index_8'] = mkt_cap_df[baseline_8].sum(axis=1)/10000000

               

In [ ]:
quarterly = determine_reweighting(us, n = 12)

### Visualization

#### Seaborn color maps:
‘Accent’, ‘Accent_r’, ‘Blues’, ‘Blues_r’, ‘BrBG’, ‘BrBG_r’, ‘BuGn’, ‘BuGn_r’, ‘BuPu’, ‘BuPu_r’, ‘CMRmap’, ‘CMRmap_r’, ‘Dark2’, ‘Dark2_r’, ‘GnBu’, ‘GnBu_r’, ‘Greens’, ‘Greens_r’, ‘Greys’, ‘Greys_r’, ‘OrRd’, ‘OrRd_r’, ‘Oranges’, ‘Oranges_r’, ‘PRGn’, ‘PRGn_r’, ‘Paired’, ‘Paired_r’, ‘Pastel1’, ‘Pastel1_r’, ‘Pastel2’, ‘Pastel2_r’, ‘PiYG’, ‘PiYG_r’, ‘PuBu’, ‘PuBuGn’, ‘PuBuGn_r’, ‘PuBu_r’, ‘PuOr’, ‘PuOr_r’, ‘PuRd’, ‘PuRd_r’, ‘Purples’, ‘Purples_r’, ‘RdBu’, ‘RdBu_r’, ‘RdGy’, ‘RdGy_r’, ‘RdPu’, ‘RdPu_r’, ‘RdYlBu’, ‘RdYlBu_r’, ‘RdYlGn’, ‘RdYlGn_r’, ‘Reds’, ‘Reds_r’, ‘Set1’, ‘Set1_r’, ‘Set2’, ‘Set2_r’, ‘Set3’, ‘Set3_r’, ‘Spectral’, ‘Spectral_r’, ‘Wistia’, ‘Wistia_r’, ‘YlGn’, ‘YlGnBu’, ‘YlGnBu_r’, ‘YlGn_r’, ‘YlOrBr’, ‘YlOrBr_r’, ‘YlOrRd’, ‘YlOrRd_r’, ‘afmhot’, ‘afmhot_r’, ‘autumn’, ‘autumn_r’, ‘binary’, ‘binary_r’, ‘bone’, ‘bone_r’, ‘brg’, ‘brg_r’, ‘bwr’, ‘bwr_r’, ‘cividis’, ‘cividis_r’, ‘cool’, ‘cool_r’, ‘coolwarm’, ‘coolwarm_r’, ‘copper’, ‘copper_r’, ‘crest’, ‘crest_r’, ‘cubehelix’, ‘cubehelix_r’, ‘flag’, ‘flag_r’, ‘flare’, ‘flare_r’, ‘gist_earth’, ‘gist_earth_r’, ‘gist_gray’, ‘gist_gray_r’, ‘gist_heat’, ‘gist_heat_r’, ‘gist_ncar’, ‘gist_ncar_r’, ‘gist_rainbow’, ‘gist_rainbow_r’, ‘gist_stern’, ‘gist_stern_r’, ‘gist_yarg’, ‘gist_yarg_r’, ‘gnuplot’, ‘gnuplot2’, ‘gnuplot2_r’, ‘gnuplot_r’, ‘gray’, ‘gray_r’, ‘hot’, ‘hot_r’, ‘hsv’, ‘hsv_r’, ‘icefire’, ‘icefire_r’, ‘inferno’, ‘inferno_r’, ‘jet’, ‘jet_r’, ‘magma’, ‘magma_r’, ‘mako’, ‘mako_r’, ‘nipy_spectral’, ‘nipy_spectral_r’, ‘ocean’, ‘ocean_r’, ‘pink’, ‘pink_r’, ‘plasma’, ‘plasma_r’, ‘prism’, ‘prism_r’, ‘rainbow’, ‘rainbow_r’, ‘rocket’, ‘rocket_r’, ‘seismic’, ‘seismic_r’, ‘spring’, ‘spring_r’, ‘summer’, ‘summer_r’, ‘tab10’, ‘tab10_r’, ‘tab20’, ‘tab20_r’, ‘tab20b’, ‘tab20b_r’, ‘tab20c’, ‘tab20c_r’, ‘terrain’, ‘terrain_r’, ‘turbo’, ‘turbo_r’, ‘twilight’, ‘twilight_r’, ‘twilight_shifted’, ‘twilight_shifted_r’, ‘viridis’, ‘viridis_r’, ‘vlag’, ‘vlag_r’, ‘winter’, ‘winter_r’

Attributes = ['address1',
 'city',
 'state',
 'zip',
 'country',
 'phone',
 'website',
 'industry',
 'industryKey',
 'industryDisp',
 'sector',
 'sectorKey',
 'sectorDisp',
 'longBusinessSummary',
 'fullTimeEmployees',
 'companyOfficers',
 'auditRisk',
 'boardRisk',
 'compensationRisk',
 'shareHolderRightsRisk',
 'overallRisk',
 'governanceEpochDate',
 'compensationAsOfEpochDate',
 'maxAge',
 'priceHint',
 'previousClose',
 'open',
 'dayLow',
 'dayHigh',
 'regularMarketPreviousClose',
 'regularMarketOpen',
 'regularMarketDayLow',
 'regularMarketDayHigh',
 'beta',
 'forwardPE',
 'volume',
 'regularMarketVolume',
 'averageVolume',
 'averageVolume10days',
 'averageDailyVolume10Day',
 'bid',
 'ask',
 'bidSize',
 'askSize',
 'marketCap',
 'fiftyTwoWeekLow',
 'fiftyTwoWeekHigh',
 'priceToSalesTrailing12Months',
 'fiftyDayAverage',
 'twoHundredDayAverage',
 'currency',
 'enterpriseValue',
 'profitMargins',
 'floatShares',
 'sharesOutstanding',
 'sharesShort',
 'sharesShortPriorMonth',
 'sharesShortPreviousMonthDate',
 'dateShortInterest',
 'sharesPercentSharesOut',
 'heldPercentInsiders',
 'heldPercentInstitutions',
 'shortRatio',
 'shortPercentOfFloat',
 'impliedSharesOutstanding',
 'bookValue',
 'priceToBook',
 'lastFiscalYearEnd',
 'nextFiscalYearEnd',
 'mostRecentQuarter',
 'netIncomeToCommon',
 'trailingEps',
 'forwardEps',
 'pegRatio',
 'enterpriseToRevenue',
 'enterpriseToEbitda',
 '52WeekChange',
 'SandP52WeekChange',
 'exchange',
 'quoteType',
 'symbol',
 'underlyingSymbol',
 'shortName',
 'longName',
 'firstTradeDateEpochUtc',
 'timeZoneFullName',
 'timeZoneShortName',
 'uuid',
 'messageBoardId',
 'gmtOffSetMilliseconds',
 'currentPrice',
 'targetHighPrice',
 'targetLowPrice',
 'targetMeanPrice',
 'targetMedianPrice',
 'recommendationMean',
 'recommendationKey',
 'numberOfAnalystOpinions',
 'totalCash',
 'totalCashPerShare',
 'ebitda',
 'totalDebt',
 'quickRatio',
 'currentRatio',
 'totalRevenue',
 'debtToEquity',
 'revenuePerShare',
 'returnOnAssets',
 'returnOnEquity',
 'freeCashflow',
 'operatingCashflow',
 'revenueGrowth',
 'grossMargins',
 'ebitdaMargins',
 'operatingMargins',
 'financialCurrency',
 'trailingPegRatio']